## 연습문제 1

다음 2건의 데이터에 대해 Merkle Root 값을 계산하고 출력하세요.

* txA = 'Hello'

* txB = 'How are you?'

In [5]:
import hashlib

tx_hA = "Hello"
tx_hB = 'How are you?'

In [6]:
#해싱, 재해싱
_hashA=hashlib.sha256(hashlib.sha256(tx_hA.encode()).digest()).hexdigest()
_hashB=hashlib.sha256(hashlib.sha256(tx_hB.encode()).digest()).hexdigest()

In [7]:
#바이트 교환
print(_hashA)
for i in range(0, hashlib.sha256(tx_hA.encode()).digest_size*2, 2):
    print(_hashA[i:i+2], end=" ")
hashlib.sha256(tx_hA.encode()).digest_size
"".join(reversed([_hashA[i:i+2] for i in range(0, hashlib.sha256(tx_hA.encode()).digest_size*2, 2)]))
hashAswap="".join(reversed([_hashA[i:i+2] for i in range(0, hashlib.sha256(tx_hA.encode()).digest_size*2, 2)]))
hashBswap="".join(reversed([_hashB[i:i+2] for i in range(0, hashlib.sha256(tx_hB.encode()).digest_size*2, 2)]))
print("hashAswap: ", hashAswap, "\nhashBswap: ",hashBswap)

70bc18bef5ae66b72d1995f8db90a583a60d77b4066e4653f1cead613025861c
70 bc 18 be f5 ae 66 b7 2d 19 95 f8 db 90 a5 83 a6 0d 77 b4 06 6e 46 53 f1 ce ad 61 30 25 86 1c hashAswap:  1c86253061adcef153466e06b4770da683a590dbf895192db766aef5be18bc70 
hashBswap:  d0e7719d7633fb945d596090dfa31a95ae81b18d90352d63fc49af7f35ce2710


In [10]:
#해시 결합, 더블 해싱, 바이트 교환
hash_hAB=hashAswap+hashBswap
hashlib.sha256(hashlib.sha256(hash_hAB.encode()).digest()).hexdigest()
_hash_hAB=hashlib.sha256(hashlib.sha256(hashAB.encode()).digest()).hexdigest()
"".join(reversed([_hash_hAB[i:i+2] for i in range(0, 32*2, 2)]))
hashABswap="".join(reversed([_hash_hAB[i:i+2] for i in range(0, 32*2, 2)]))

## 연습문제 2

친구에게  주소를 구하고, 그 주소로 송금해 보자. 송금이 되지 않으면 왜 안되는지 이유를 알아보자.

안되면 친구의 주소를 만들고 전송한다.

잔고의 증가분을 출력하세요.

소요된 gas비용 출력하세요.

In [15]:
#계정 출력
!geth --exec "eth.accounts" attach http://localhost:8445

["0x6811ba5d2ebf9a4500f3d4e9bf1d98ea0d2bb6cb", "0xe03bca5e9d19fd12d5a5ab3bcc5cc2cfc54dc42a"]


In [18]:
#내 계정 출력
!geth --exec "eth.accounts[0]" attach http://localhost:8445

"0x6811ba5d2ebf9a4500f3d4e9bf1d98ea0d2bb6cb"


In [19]:
#친구 계정 출력
!geth --exec "eth.accounts[1]" attach http://localhost:8445

"0xe03bca5e9d19fd12d5a5ab3bcc5cc2cfc54dc42a"


In [20]:
#현재 친구 잔고
!geth --exec "eth.getBalance(eth.accounts[1])" attach http://localhost:8445

5.2771875e+21


In [13]:
%%writefile src/e_testTran.js
miner.setEtherbase(eth.accounts[0]);
console.log('coinbase: ', eth.coinbase);
var bal1=eth.getBalance(eth.coinbase);
var bal2=eth.getBalance(eth.accounts[1]);
console.log('sender balance in ether: ', web3.fromWei(bal1,"ether"));
console.log('receiver balance in ether: ', web3.fromWei(bal2,"ether"));
console.log('median gas price: ', eth.gasPrice);
console.log('block number: ', eth.blockNumber);
console.log('transaction count: ', eth.getTransactionCount(eth.coinbase));
var t = eth.sendTransaction({from:eth.accounts[0], to:eth.accounts[1],value:50000});
console.log('...mining start');
miner.start(1);admin.sleepBlocks(1);miner.stop();
console.log('mining done...');
var bal1new=eth.getBalance(eth.coinbase);
var bal2new=eth.getBalance(eth.accounts[1]);
console.log('- new sender balance in ether: ', web3.fromWei(bal1new,"ether"));
console.log('- new receiver balance in ether: ', web3.fromWei(bal2new,"ether"));
console.log('- block number: ', eth.blockNumber);
console.log('- transaction count: ', eth.getTransactionCount(eth.coinbase));
console.log('gasUsed : ', eth.getTransactionReceipt(t).gasUsed);

Overwriting src/e_testTran.js


In [14]:
!geth --exec "loadScript('src/e_testTran.js')" attach http://localhost:8445

coinbase:  0x6811ba5d2ebf9a4500f3d4e9bf1d98ea0d2bb6cb
sender balance in ether:  9349.99999999999987
receiver balance in ether:  5277.18750000000013
median gas price:  1000000000
block number:  3029
transaction count:  9
...mining start
mining done...
- new sender balance in ether:  9354.99999999999967
- new receiver balance in ether:  5277.18750000000033
- block number:  3030
- transaction count:  13
gasUsed :  21000
null


In [15]:
#친구 계정의 변화
!geth --exec "eth.getBalance(eth.accounts[1])" attach http://localhost:8445

5.27718750000000033e+21


## 연습문제 3

블록헤더 데이터의 해시 값에 NONCE를 증가시키면서

앞 자리의 0의 개수를 맞출 때까지 반복한다.

(1) 찾고자 하는 해시가 ```0000```로 시작한다고 하자. 몇 회만에 찾는지 출력하세요.

(2) 찾고자 하는 해시가 ```00000```로 시작한다고 하자. 몇 회만에 찾는지 출력하세요.

(3) 찾고자 하는 해시가 ```000000```로 시작한다고 하자. 몇 회만에 찾는지 출력하세요.

In [11]:
#1번
import hashlib
ntry=1
found=False
blockNumber=54 # hex
NONCE=0
data='Hello'
previousHash='5d7c7ba21cbbcd75d14800b100252d5b428e5b1213d27c385bc141ca6b47989e'
while found==False:
    z=str(blockNumber)+str(NONCE)+data+previousHash
    guessHash=hashlib.sha256(z.encode('utf-8')).hexdigest()
    if guessHash[:4]=='0000':
        found=True
    NONCE+=1
    if(NONCE%10000000==0):   #print guessHash every 10000000
        print("NONCE: ",NONCE, guessHash)
print("Solved ", "NONCE: ", NONCE, "guessHash: ", guessHash)

Solved  NONCE:  94280 guessHash:  000043ce4a61d02bff0e68ba18a7daf448cb3b93691fdd4850f6cd3f85b7a13f


In [12]:
#2번
import hashlib
ntry=1
found=False
blockNumber=54 # hex
NONCE=0
data='Hello'
previousHash='5d7c7ba21cbbcd75d14800b100252d5b428e5b1213d27c385bc141ca6b47989e'
while found==False:
    z=str(blockNumber)+str(NONCE)+data+previousHash
    guessHash=hashlib.sha256(z.encode('utf-8')).hexdigest()
    if guessHash[:5]=='00000':
        found=True
    NONCE+=1
    if(NONCE%10000000==0):   #print guessHash every 10000000
        print("NONCE: ",NONCE, guessHash)
print("Solved ", "NONCE: ", NONCE, "guessHash: ", guessHash)

Solved  NONCE:  315753 guessHash:  000007f9f69a43f1bb6ab92672d873b93d6bafaa2007e44b6151bd19efadf4d1


In [13]:
#3번
import hashlib
ntry=1
found=False
blockNumber=54 # hex
NONCE=0
data='Hello'
previousHash='5d7c7ba21cbbcd75d14800b100252d5b428e5b1213d27c385bc141ca6b47989e'
while found==False:
    z=str(blockNumber)+str(NONCE)+data+previousHash
    guessHash=hashlib.sha256(z.encode('utf-8')).hexdigest()
    if guessHash[:6]=='000000':
        found=True
    NONCE+=1
    if(NONCE%10000000==0):   #print guessHash every 10000000
        print("NONCE: ",NONCE, guessHash)
print("Solved ", "NONCE: ", NONCE, "guessHash: ", guessHash)

NONCE:  10000000 660b9e057377381579f5c54347901cf462fce656c069a4d0f26bdd4cf1e05e66
NONCE:  20000000 c31d3ddd7bb92312bcc2a88263d92e94c6d17a641ac9e59ac262c775c7f58925
NONCE:  30000000 236077716f4ce428ec83f12bf74fb7ab76450bad60bc3f21d924e200c39a9fa4
NONCE:  40000000 5b07929bd7088f11d6caf0e65a6f44ab3b7f82d32900430cc7c7244b2917e04d
Solved  NONCE:  45576417 guessHash:  0000003d02b95604bb1ec436ff20e08168dd339f2ec0f9941bfc58bad039994e


# 결론
난이도는 세번째가 가장 높았다. 따라서 NONCE의 값이 높을수록 난이도가 높다.